# README
Run this notebook to create the dataset for mosaic training.

In [1]:
from torch.utils.data import Dataset as BaseDataset
import albumentations as albu

import copy
import random
import time
from functools import wraps

import torch
from pathlib import Path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from tqdm import tqdm
import logging

# np.random.seed(42)
# random.seed(42)

In [2]:
def get_patch_label(file):
    # [a: Tumor epithelial (TE), b: Necrosis (NEC), c: Lymphocyte (LYM), d: Tumor-associated stroma (TAS)]
    if isinstance(file, Path):
        file = str(file)
    fname = file[:-4]
    label_str = '[' + fname.split(']')[0].split('[')[-1] + ']'
    label_str = label_str.replace(' ', ',')
    label = eval(label_str)
    return label

In [3]:
train_image_dir = Path('data/LUAD-HistoSeg/train/')
train_image_list = sorted(list(train_image_dir.glob('*.png')))
len(train_image_list)

16678

In [4]:
only_tum_num = 0
only_nec_num = 0
only_lym_num = 0
only_tas_num = 0
single_type_num = 0

only_tum_list = []
only_nec_list = []
only_lym_list = []
only_tas_list = []

for train_image in train_image_list:
    big_label = get_patch_label(train_image)
    if np.sum(big_label) == 1:
        single_type_num += 1
        if big_label[0] == 1:
            only_tum_num += 1
            only_tum_list.append(train_image)
        elif big_label[1] == 1:
            only_nec_num += 1
            only_nec_list.append(train_image)
        elif big_label[2] == 1:
            only_lym_num += 1
            only_lym_list.append(train_image)
        elif big_label[3] == 1:
            only_tas_num += 1
            only_tas_list.append(train_image)
            
assert single_type_num == only_tum_num + only_nec_num + only_lym_num + only_tas_num
assert len(only_tum_list) == only_tum_num
assert len(only_nec_list) == only_nec_num
assert len(only_lym_list) == only_lym_num
assert len(only_tas_list) == only_tas_num


In [5]:
print(only_tum_num, only_nec_num, only_lym_num, only_tas_num)
print(single_type_num)

1574 787 42 2192
4595


----

In [6]:
class CropAndConcatDataset(BaseDataset):    
    def __init__(self, args, patch_num, patch_size, size=None):
       
        self.args = args
        self.train_dir = args.train_dir
        self.train_images = list(Path(self.train_dir).glob('*.png'))
        self.tum, self.nec, self.lym, self.tas = only_tum_list, only_nec_list, only_lym_list, only_tas_list
        self.single_type_images = self.tum + self.nec + self.lym + self.tas

        self.patch_num = patch_num
        self.patch_size = patch_size

        self.total_len = len(self.single_type_images) if size is None else size

        self.crop_fn = albu.Compose([
            albu.PadIfNeeded(min_height=self.patch_size, min_width=self.patch_size),
            albu.RandomCrop(width=self.patch_size, height=self.patch_size)
        ])
        
        self.transform = albu.Compose([
            albu.Flip(),
            albu.RandomRotate90(),
        ])
    
    def __getitem__(self, i):
        # np.random.seed(2022 + 2022 * i)
        # random.seed(2022 + 2022 * i)
        
        H = W = self.patch_num * self.patch_size
        
        while True:
            try:
                (image_1, mask_1), (image_2, mask_2), (image_3, mask_3), (image_4, mask_4) = [self.create_one_image() for _ in range(4)] # [H, W, C]
                image, mask = self.create_mosaic(H, W, image_1, mask_1, image_2, mask_2, image_3, mask_3, image_4, mask_4)
                break            
            except AssertionError as e:
                print(e)

        
        return image, mask

    def create_one_image(self):      
        H = W = self.patch_num * self.patch_size
        image = np.zeros((H, W, 3), dtype=np.uint8)
        mask = np.zeros((H, W), dtype=np.uint8)

        for i in range(self.patch_num):
            for j in range(self.patch_num):

                tile_name = np.random.choice(self.single_type_images)
                label = get_patch_label(tile_name)
                assert sum(label) == 1
                tile = np.asarray(Image.open(tile_name))
                label = label.index(1)
                tile_mask = np.full((tile.shape[0], tile.shape[1]), label)

                sample = self.crop_fn(image=tile, mask=tile_mask)
                tile = sample['image']
                tile_mask = sample['mask']
                
                image[i*self.patch_size: (i+1)*self.patch_size, j*self.patch_size: (j+1)*self.patch_size] = tile
                mask[i*self.patch_size: (i+1)*self.patch_size, j*self.patch_size: (j+1)*self.patch_size] = tile_mask
        
        return image, mask
    
    def create_mosaic(self, H, W, image_1, mask_1, image_2, mask_2, image_3, mask_3, image_4, mask_4):
        def get_transforms(height, width, p=0.5):
            _transform = [
                albu.Flip(p=p),
                albu.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=p),
                albu.RandomCrop(height, width),
            ]
            return albu.Compose(_transform)
        
        image = np.zeros((H, W, 3), dtype=np.uint8)
        mask = np.zeros((H, W), dtype=np.uint8)

        h, w = int(H * (random.random()*0.6+0.2)), int(W * (random.random()*0.6+0.2))
        h += h % 2
        w += w % 2

        transform_1 = get_transforms(height=h, width=w, p=0.8)
        sample = transform_1(image=image_1, mask=mask_1)
        image_1, mask_1 = sample['image'], sample['mask']

        transform_2 = get_transforms(height=h, width=W-w, p=0.8)
        sample = transform_2(image=image_2, mask=mask_2)
        image_2, mask_2 = sample['image'], sample['mask']

        transform_3 = get_transforms(height=H-h, width=w, p=0.8)
        sample = transform_3(image=image_3, mask=mask_3)
        image_3, mask_3 = sample['image'], sample['mask']

        transform_4 = get_transforms(height=H-h, width=W-w, p=0.8)
        sample = transform_4(image=image_4, mask=mask_4)
        image_4, mask_4 = sample['image'], sample['mask']
        
        image[:h, :w, :] = image_1
        image[:h, w:W, :] = image_2
        image[h:H, :w, :] = image_3
        image[h:H, w:W, :] = image_4
    
        mask[:h, :w] = mask_1
        mask[:h, w:W] = mask_2
        mask[h:H, :w] = mask_3
        mask[h:H, w:W] = mask_4
        
        return image, mask

    def __len__(self):
        return self.total_len

In [7]:
from argparse import Namespace
from torch.utils.data import DataLoader
args = Namespace()
args.train_dir = "./data/LUAD-HistoSeg/train"

# set to 2 and 112 for better performance
patch_num = 2
patch_size = 112
N=10_000


In [8]:
from joblib import Parallel, delayed

for run in range(1, 5):
    mosaic_data = Path(f"./data/LUAD-HistoSeg/mosaic_{patch_num}_{patch_size}_run{run}")
    (mosaic_data / 'img').mkdir(parents=True, exist_ok=True)
    (mosaic_data / 'mask').mkdir(parents=True, exist_ok=True)

    dataset = CropAndConcatDataset(args, patch_num=patch_num, patch_size=patch_size, size=N)

    def func(i):
        image, mask = dataset[i]
        image = Image.fromarray(image)
        palette = [0]*15
        palette[0:3] = [205,51,51]          # Tumor epithelial (TE)
        palette[3:6] = [0,255,0]            # Necrosis (NEC)
        palette[6:9] = [65,105,225]         # Lymphocyte (LYM)
        palette[9:12] = [255,165,0]         # Tumor-associated stroma (TAS)
        palette[12:15] = [255, 255, 255]    # White background or exclude
        mask = Image.fromarray(np.uint8(mask), mode='P')
        mask.putpalette(palette)
        image.save(mosaic_data / 'img' / f'{patch_num}_{patch_size}_{i}.png')
        mask.save(mosaic_data / 'mask' / f'{patch_num}_{patch_size}_{i}.png')
    def print_error(value):
        print("error: ", value)

    Parallel(n_jobs=4)(delayed(func)(i) for i in tqdm(range(N)));

100%|██████████| 10000/10000 [13:16<00:00, 12.55it/s]
